<div style="text-align: center;">
<span style="color:#C1D3FE;font-size: 32px; font-weight: bold;">Notebook de Treinamento de Machine Learning: "Originadora e StartCard" de 05/08</span>
<br> <span style="color:#FFD1DC;"> Feito por Lucas Andrade entre 21 e 22 de Agosto de 2025 </span>
<br> <span style="color:#FFF5BA;"> a ser revisado por Felipe Bastos

<br>
<span style="color:#C1D3FE;">Lucas Rafael de Andrade Desenvolvimento de Software Ltda (59.088.726/0001-70)</span>

<span style="color:#C1D3FE;">Solicitado por Porto Real Asset</span>


Observação: esse notebook traz como features para o modelo as seguintes séries: 
Esfera do convênio,
Faixa população,
CAPAG,
Muitos contratos,
Tipo de produto,
Tipo de empregado,
Faixa de Idade,
Total de parcelas,
Valor da parcela

Falta incluir: 
Dados econômicos (como PIB e População), e # prox parcela

### Bibliotecas

In [14]:
# -*- coding: utf-8 -*-
"""
VERSãO: FALTANDO COLOCAR DADOS COMO POPULAçÂO E PIB PER CAPITA
"""
#* baza >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
import pandas as pd
import numpy as np
import os
import warnings
#* modeloj <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
#* Metriko >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
from sklearn.metrics import roc_auc_score, average_precision_score
#* Antaŭprilaborado <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
from sklearn.impute import SimpleImputer

#! CUIDADO! Isso pode ser útil )))))))))))))))))))))))))))))))
warnings.filterwarnings('ignore')
#! )))))))))))))))))))))))))))))))))))))))))))))))))))))))))))


### Função de Carregamento dos dados e preparação 
retorna um df com  as features, já

In [15]:

# =============================================================================
#  FUNÇÂO:
#  CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================
def carregar_e_preparar_dados_corrigido():
    """
    por enquanto lê só  StarCard.xlsx e Originadores.xlsx
     Uso 'Data de Inclusão' para ordenar o seguinte
        - duplicacoes de ccb
        - _MuitosContratos c/ cum Count (para evitrar leak)
        - Idade calculada na Data de Inclusão (fallback em DataGeracao se der m)
    Crio labels 'Vencido_{1,30,60,90}d' 
    """
    print("="*88)
    print("ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...")
    print("="*88)

    #! PATHS ----------------------------------------------------------------------
    #! COLOQUE CORRETAMENTe na maquina 
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    #!------------------------------------------------------------------------------

    #******************   Leitura       *************************************************
    df_starcard = pd.read_excel(path_starcard)
    cols_originadores = ['CCB', 'Data de Inclusão', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio', 'CAPAG']
    # CAPAG pode não existir; vamos ler com try simples
    try:
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)
    except ValueError:
        # <deprec> debug do AC

        print("[ALERTA] Coluna 'CAPAG' não encontrada em Originadores.xlsx. Esta feature será desconsiderada no modelo.")
        cols_originadores = [c for c in cols_originadores if c != 'CAPAG']
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

    #* Normalizo garantindo que vai pegar por inlcusao na ordem
    if 'Data de Inclusão' in df_originadores.columns:
        df_originadores['Data de Inclusão'] = pd.to_datetime(df_originadores['Data de Inclusão'], errors='coerce')
        df_originadores = (
            df_originadores
            .sort_values(['CCB', 'Data de Inclusão'])
            .drop_duplicates(subset='CCB', keep='first')
        )
    """faco isso para que ele pegue de forma deterinística a mesma versao da ccb"""

    #* Merge: 
    #* note que o startcard repete os ccbs em varias linhas e originadores tem apenas uma lina por ccb 
    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

   
    df_merged = df_merged.rename(columns={  # Renomeios #*(para ser compatível com fct_consig- outros dados)
        'Data Referencia': 'DataGeracao',
        'Data Vencimento': 'DataVencimento',
        'ID Cliente': 'SacadoID'
    }) 
    #?----------------------------------------------------------------
    for col in ['DataGeracao', 'DataVencimento', 'Data de Nascimento']:
        if col in df_merged.columns:
            df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')     # Tipos de data
    #?----------------------------------------------------------------
    # Conversão  #? sim, é necessário #----------------------------------------------------------------
    if 'Valor Parcela' in df_merged.columns and df_merged['Valor Parcela'].dtype == 'object':
        s = df_merged['Valor Parcela'].astype(str)
        s = s.str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged['Valor Parcela'] = pd.to_numeric(s, errors='coerce')
    #?----------------------------------------------------------------
    df_base = df_merged.copy() # agora vamos usar esse df
    # cleanup da memoria
    del df_starcard, df_originadores, df_merged
    #?----------------------------------------------------------------
    # Limpezas básicas
    if 'Produto' in df_base.columns:
        df_base['Produto'] = df_base['Produto'].fillna('')
    if 'Convênio' in df_base.columns:
        df_base['Convênio'] = df_base['Convênio'].fillna('')
    #?----------------------------------------------------------------
    # feat dias_de_atraso
    if 'DataGeracao' in df_base.columns and 'DataVencimento' in df_base.columns:
        df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days
    #?----------------------------------------------------------------
    # buckets de produto/empregado/convênio
    condicoes_produto = [
        df_base['Produto'].str.contains('Empréstimo', case=False, na=False),
        df_base['Produto'].str.contains('Cartão RMC', case=False, na=False),
        df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)
    ]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')
    """Agrupar em buckets transforma informação
    não estruturada em features úteis e reduz a complexidade 
    para os modelos"""

    #?----------------------------------------------------------------
    condicoes_empregado = [
        df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True),
        df_base['Produto'].str.contains('Temporário', case=False, na=False),
        df_base['Produto'].str.contains('CONTRATADO', case=False, na=False),
        df_base['Produto'].str.contains('Comissionado', case=False, na=False)
    ]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')
    #?----------------------------------------------------------------
    # Rgx do convênio
    condicoes_convenio = [
        df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True),
        df_base['Convênio'].str.contains(r'PREF\.|PRERF', case=False, na=False, regex=True)
    ] #* note que eu ajustei para o typo que observei empiricamente em pref
    opcoes_convenio = ['Estadual', 'Municipal']
    df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')

    # ----- Data de Inclusão -----
    """
    explicação do que eu fiz abaixo: 
    tentei evitar vazamento de dados do futuro. 
    A feature _MuitosContratos reflete o histórico
    do cliente até este momento, e nao o total de contratos
    q ele viria a ter. 
    Da mesma forma, 
    a idade é a que ele tinha quando assinou o contrato,
     que eh a informação que estaria disponivel 
     para uma analise de credito na época."""
    if 'Data de Inclusão' not in df_base.columns:
        raise ValueError("Coluna 'Data de Inclusão' não encontrada após o merge. Verifique Originadores.xlsx.")
    #?----------------------------------------------------------------
    # ordeno por inclusão para _MuitosContratos
    df_base = df_base.sort_values(['SacadoID', 'Data de Inclusão', 'CCB'])
    df_base['_NumContratosAnteriores'] = df_base.groupby('SacadoID').cumcount()
    df_base['_MuitosContratos'] = (df_base['_NumContratosAnteriores'] >= 2).astype(int)
    #?----------------------------------------------------------------
    # Idade na Data de Inclusão 
    # (fallback em DataGeracao, caso desse errado)
    if 'Data de Nascimento' in df_base.columns:
        base_idade = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
        df_base['_IdadeCliente'] = (base_idade - df_base['Data de Nascimento']).dt.days / 365.25
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)
    #? ---------- Labels por CCB ----------
    #? Se tivesse múltiplas linhas por CCB (parcelas/linhas), agregaria o maior atraso
    #? <deprec>
    #if 'dias_de_atraso' not in df_base.columns:
    #    raise ValueError("'dias_de_atraso' não foi criado. Verifique 'DataGeracao' e 'DataVencimento'.")

    """explicação de abaixo: 
    o que eu faço é deixar a variável que eu quero prever como unequívoca 
    para ter como objetivo e ser explicada, para ter a mesma dimensionalidade entr
    o contrato e o n de linhas """
    atraso_por_ccb = (
        df_base.groupby('CCB', as_index=False)['dias_de_atraso'].max()
        .rename(columns={'dias_de_atraso': 'max_dias_atraso'})
    )
    for k in [1, 30, 60, 90]:
        atraso_por_ccb[f'Vencido_{k}d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= k).astype(int)

    # snapshot pela ordem da nclusao
    df_base['DataInclRef'] = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
    snapshots = df_base.sort_values(['CCB', 'DataInclRef', 'DataVencimento'])
    primeiro_snap = snapshots.groupby('CCB', as_index=False).first()
    #! ########################################################################################
    #! #####################     IMPORTANTE     ###############################################
    features_para_modelo = [
        'CCB',  'CAPAG', 'Prazo', 'Valor Parcela', #'Convênio',   ## comente convênio se quisr remver
        '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins', '_MuitosContratos'
    ]
    #! #########################################################################################
    ausentes = sorted(list(set(features_para_modelo) - set(primeiro_snap.columns)))
    if ausentes: # dbg AC
        print(f"[AVISO] Features ausentes e removidas (não encontradas no snapshot): {ausentes}")

    features_existentes = [f for f in features_para_modelo if f in primeiro_snap.columns]
    df_features = primeiro_snap[features_existentes]

    # df_ml final #? esse vai ser o df usado no modelo de ML, mesmo
    df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB', how='inner').rename(columns={
        'Prazo': 'Total_Parcelas',
        'Valor Parcela': 'Valor_Parcela',
        '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade',
        '_MuitosContratos': 'Muitos_Contratos',
        '_TipoProduto': 'Tipo_Produto',
        '_TipoEmpregado': 'Tipo_Empregado'
    })

    #? verificações básicas
    #?assert df_ml['CCB'].is_unique, "CCB não está único no df_ml! Revise o snapshot por CCB."
    #?assert df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].notna().all().all(), \
    #?    "Algum label Vencido_* está com NaN."
    
    print(f"\n <DBG> Dataset para ML criado com {df_ml.shape[0]} linhas (CCBs) e {df_ml.shape[1]} colunas.")
    return df_ml



### Função De Treino e Avaliação

In [16]:

# =============================================================================
# FUNÇÃO: 
#   TREINAMENTO E AVALIAÇÃO
# =============================================================================
def treinar_e_avaliar(df_ml):
    targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
    resultados_finais = pd.DataFrame()

    for target in targets:
        print("\n" + "="*88)
        print(f"   TREINANDO MODELOS PARA O ALVO: {target}")
        print("="*88)
        #?----------------------------------------------------------------    
        # Def X e y  #?(OBS: Removo colunas desncessárias para o train)
        X = df_ml.drop(columns=targets + ['CCB', 'max_dias_atraso'])
        y = df_ml[target]
        #?-----------------  tipos: -----------------------------------------------
        numerical_features = X.select_dtypes(include=np.number).columns.tolist()
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        #?-------------------  split: ---------------------------------------------        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.30, random_state=42, stratify=y # OBS:  Split estratificado
        )
        """ note que é importante tentar fazer o split o mais rápido possível,
        para trabalhar apenas com os padrões que ele pegar no conj de teste, para 
        garantir que nao tenha nenhuma contaminação """

        #?---------- torno Dummy (após o split para nao contaminar) ------------------------------------------------------
        X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
        X_test  = pd.get_dummies(X_test,  columns=categorical_features, drop_first=True)
        X_test  = X_test.reindex(columns=X_train.columns, fill_value=0)
        #?------------------- <debug> ---------------------------------------------
        #print("#cols treino:", X_train.shape[1], "| #cols teste:", X_test.shape[1])
        #assert list(X_train.columns) == list(X_test.columns), "Treino e teste desalinhados!"
        #?------------------ Imputação + Escalonamento  ----------------------------------------------
        # só nas nas numéricas originais (interseção)
        #* pelo que eu pesquisei o scalling não é necessário, mas vou deixar para se quiser 
        #* usar outros modelos, além de que para a reg logistica é útil 
        num_cols = [c for c in numerical_features if c in X_train.columns]
        #? if len(num_cols) > 0: # idente abaixo para <bebug>
        imputer = SimpleImputer(strategy='median')
        X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
        X_test[num_cols]  = imputer.transform(X_test[num_cols])
        #?-----------------------
        scaler = StandardScaler()
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_test[num_cols]  = scaler.transform(X_test[num_cols])

        #?else:
        #?    print("[INFO] Nenhuma coluna numérica foi identificada para imputação ou escalonamento.")

        # Log base rates # - coloco isso para estudar o balanceio
        print("Base rate no treino:", float(y_train.mean()).__round__(4), "| no teste:", float(y_test.mean()).__round__(4))
        #?-----------------------------
        # scale_pos_weight com guarda

        """
        note que temos um debalanceio muito grande nesse tipo de porblema, fazendo 
        com que o treinamento fique viesado para um lado. 
        Isso aqui faz comm que ele de mais peso à classe minoritária
        #? obs: o paralelo aqui é o de fraudes em cartão de crédito
        """
        pos = int((y_train == 1).sum())
        neg = int((y_train == 0).sum())
        if pos == 0:
            print(f"[AVISO] Classe positiva ausente no treino para {target}. Ajustando scale_pos_weight=1.0.")
            scale_pos_weight = 1.0
        else:
            scale_pos_weight = neg / pos
        #?---------------- ------------------------------------------------
        #?---------------- MODELOS -----------------------------------------------
        #! ####################################################################################################
        models = {
            "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
            "Random Forest":       RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
            "SVM (Kernel RBF)":    SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced'),
            "XGBoost":             xgb.XGBClassifier(random_state=42, eval_metric='logloss',
                                                      use_label_encoder=False, scale_pos_weight=scale_pos_weight,
                                                      n_jobs=-1),
            "LightGBM":            lgb.LGBMClassifier(random_state=42, scale_pos_weight=scale_pos_weight,
                                                       n_jobs=-1)
        }
        #! ######################################################################################################

        resultados_target = {}
        for name, model in models.items():
            print(f"\n--- Treinando {name} ---")
            model.fit(X_train, y_train)
            #?---------------- probabilidades -----------------------------------------------
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            #?----------------metricas -----------------------------------------------
            auc = roc_auc_score(y_test, y_pred_proba)
            ap  = average_precision_score(y_test, y_pred_proba)
            resultados_target[name] = {'ROC-AUC': auc, 'PR-AUC(AP)': ap}
            print(f"ROC-AUC: {auc:.4f} | PR-AUC(AP): {ap:.4f}")
            #?---------------- IMPORTANCIAS -----------------------------------------------
            """note que o objetivo é explicar, entao isso é muito importante 
            o motivo de usar isso está nas referÊncias"""
            if isinstance(model, xgb.XGBClassifier):
                booster = model.get_booster()
                imp = booster.get_score(importance_type='gain')
                if len(imp) > 0:
                    importances = (pd.Series(imp).sort_values(ascending=False).head(10)
                                   .rename('Gain').reset_index().rename(columns={'index': 'Feature'}))
                    print("Top 10 importâncias (XGB, gain):")
                    print(importances)
            elif isinstance(model, lgb.LGBMClassifier):
                feats = model.feature_name_
                gains = model.booster_.feature_importance(importance_type='gain')
                importances = (pd.DataFrame({'Feature': feats, 'Gain': gains})
                               .sort_values('Gain', ascending=False).head(10))
                print("Top 10 importâncias (LGBM, gain):")
                print(importances)
            elif hasattr(model, 'feature_importances_'):
                importances = pd.DataFrame({
                    'Feature': X_train.columns,
                    'Importance': model.feature_importances_
                }).sort_values('Importance', ascending=False).head(10)
                print("Top 10 importâncias:")
                print(importances)
            elif isinstance(model, LogisticRegression):
                # A comparação da magnitude destes coeficientes é válida porque 
                # as features numéricas foram escalonadas.
                coefs = pd.Series(model.coef_[0], index=X_train.columns).sort_values(key=np.abs, ascending=False).head(10)
                print("Top 10 |coef| (Regressão Logística):")
                print(coefs)

        #? Acumula resultados por alvo
        df_resultados_target = (pd.DataFrame(resultados_target).T)
        #? guarda  ROC-AUC numa tabela comparativa 

        
        resultados_finais = pd.concat([resultados_finais, df_resultados_target['ROC-AUC'].rename(target)], axis=1)

        #? Checagens adicionais úteis
        #?suspeitas = {'dias_de_atraso', 'max_dias_atraso', 'DataGeracao', 'DataVencimento', 'Data de Inclusão'}
        #?suspeitas_presentes = sorted(list(set(X_train.columns) & suspeitas))
        #?if suspeitas_presentes:
        #?    print(f"[ALERTA] Colunas suspeitas presentes em X: {suspeitas_presentes}")

    return resultados_finais




### Execução

#### Ler dados

In [17]:
#! demorado (até 4m30s)
df_ml = carregar_e_preparar_dados_corrigido()

print("\nBase rates dos alvos (df_ml):") # ( relatório rápido de base rates por label)
print(df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].mean().round(4))


ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...

 <DBG> Dataset para ML criado com 6838 linhas (CCBs) e 14 colunas.

Base rates dos alvos (df_ml):
Vencido_1d_Flag     0.8157
Vencido_30d_Flag    0.6897
Vencido_60d_Flag    0.5294
Vencido_90d_Flag    0.3353
dtype: float64


#### Treinar Modelos e Avaliar

<span style="color:#FFD1DC;">Contém Markdowns com TL;DR </span>

In [18]:
print (" TREINANDO MODELOS... AGUARDE (+-1 MIN)")
resultados_finais = treinar_e_avaliar(df_ml)


 TREINANDO MODELOS... AGUARDE (+-1 MIN)

   TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag
Base rate no treino: 0.8157 | no teste: 0.8158

--- Treinando Regressão Logística ---
ROC-AUC: 0.9112 | PR-AUC(AP): 0.9677
Top 10 |coef| (Regressão Logística):
Tipo_Empregado_Contratado   -4.139504
Tipo_Empregado_Efetivo       3.322750
Esfera_Convenio_Municipal   -2.938538
Tipo_Empregado_Temporário    2.891798
CAPAG_B                     -1.461829
CAPAG_N.D.                  -1.403280
Tipo_Produto_Empréstimo     -1.209099
Tipo_Empregado_Outros        0.695386
CAPAG_C                     -0.161676
Tipo_Produto_Cartão RMC     -0.140456
dtype: float64

--- Treinando Random Forest ---
ROC-AUC: 0.9375 | PR-AUC(AP): 0.9807
Top 10 importâncias:
                      Feature  Importance
1               Valor_Parcela    0.220520
0              Total_Parcelas    0.207176
8   Tipo_Empregado_Contratado    0.161193
11  Tipo_Empregado_Temporário    0.124871
12  Esfera_Convenio_Municipal    0.059075
9      Tipo

O Fator mais importante:  (`Tipo_Empregado`) 

Este foi, de longe, o fator mais impactante em quase todos os modelos.
* **Menor Risco:** Clientes com vínculo `Contratado` consistentemente representam um risco muito menor.
* **Maior Risco:** Clientes com vínculos `Efetivo` e `Temporário` estão associados a um risco significativamente maior.

Valor e Prazo da Parcela

As características do contrato em si parecem ser relevantes, especialmente para modelos como Random Forest e LightGBM.
* `Valor_Parcela` e `Total_Parcelas` apareceram consistentemente no topo. Quanto maior a dívida, maior o risco. A importância do `Valor_Parcela` cresce drasticamente ao prever inadimplências mais longas (90 dias).

Contexto do Contrato :


Fatores que descrevem o ambiente do contrato também têm sua importância explicativa
* **`Esfera_Convenio_Municipal`:** Ser de um convênio municipal se mostrou um fator de proteção, reduzindo o risco.
* **`Tipo_Produto_Empréstimo`:** Contratos de Empréstimo são, em geral, mais seguros que os de Cartão.
* **`CAPAG`:** A classificação de capacidade de pagamento também apareceu como um preditor relevante em diversos cenários.

(Pré) Conclusão: 
1.   O **LightGBM** parece ser o modelo mais preciso e robusto para todas as faixas de previsão de inadimplência.
2.  A principal variável para entender o risco é o **vínculo empregatício** do cliente.
3.  A melhor previsão combina o **perfil do cliente** (vínculo) com os **dados do contrato** (valor, prazo, produto) e o **contexto** (esfera do convênio)- logo é multifatores

#### Comparativo:

In [19]:

print("\n" + "="*88)
print("   RESUMO COMPARATIVO FINAL - ROC-AUC")
print("="*88)
print(resultados_finais.round(4))

melhor_modelo_geral = resultados_finais.mean(axis=1).idxmax()
print("\n" + "-"*88)
print(f"Considerando a média de ROC-AUC em todos os alvos, o melhor modelo parece ser: '{melhor_modelo_geral}'")
print("-"*88)


   RESUMO COMPARATIVO FINAL - ROC-AUC
                     Vencido_1d_Flag  Vencido_30d_Flag  Vencido_60d_Flag  \
Regressão Logística           0.9112            0.8480            0.8513   
Random Forest                 0.9375            0.8822            0.8663   
SVM (Kernel RBF)              0.9116            0.8336            0.8541   
XGBoost                       0.9337            0.8901            0.8790   
LightGBM                      0.9400            0.8953            0.8856   

                     Vencido_90d_Flag  
Regressão Logística            0.7809  
Random Forest                  0.7975  
SVM (Kernel RBF)               0.7841  
XGBoost                        0.8290  
LightGBM                       0.8371  

----------------------------------------------------------------------------------------
Considerando a média de ROC-AUC em todos os alvos, o melhor modelo parece ser: 'LightGBM'
------------------------------------------------------------------------------------

### Stats Models

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)  #! !!!!! 

# TRATAR DADOS 
df_sm = df_ml.copy()
df_sm.columns = df_sm.columns.str.replace(' ', '_', regex=False).str.replace('[^A-Za-z0-9_]+', '', regex=True)
#(ele nao parece aceitar espaco, ç, etc)
df_sm.dropna(inplace=True) # para garantir, nao deve fazer dfrença
print("="*80)
print("Nomes das colunas limpos para o statsmodels:")
print(df_sm.columns.tolist())
print("="*80)

# REGRESSÃO LINEAR (OLS)
# OBJETIVO: Entender quais fatores influenciam a QUANTIDADE de dias em atraso.
# como é reg linear, nao adianta tentar chegar a fatos binários, e sim a quantidades

print("\n### 1. Regressão Linear (OLS) para 'max_dias_atraso' ###\n")
target_ols = 'max_dias_atraso'
categorical_features = df_sm.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
numeric_features = df_sm.select_dtypes(include=np.number).columns.tolist() # identifico features

#? ---------- removo colunas inúteis, como o próprio alvo, ou códigos --------------------------------------------------------------------------------
predictors = [col for col in df_sm.columns if col not in ['CCB', 'max_dias_atraso'] + [c for c in df_sm.columns if 'Vencido' in c]]
formula_parts = []
for col in predictors:
    if col in categorical_features:
        formula_parts.append(f"C({col})")
    else:
        formula_parts.append(col)
        
formula_ols = f"{target_ols} ~ " + " + ".join(formula_parts)

print(f"Fórmula utilizada:\n{formula_ols}\n")

# TREINAMENTO
model_ols = smf.ols(formula=formula_ols, data=df_sm).fit()

#SUMÁRIO
print(model_ols.summary())



Nomes das colunas limpos para o statsmodels:
['CCB', 'CAPAG', 'Total_Parcelas', 'Valor_Parcela', 'Tipo_Produto', 'Tipo_Empregado', 'Esfera_Convenio', 'Faixa_Idade', 'Muitos_Contratos', 'max_dias_atraso', 'Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']

### 1. Regressão Linear (OLS) para 'max_dias_atraso' ###

Fórmula utilizada:
max_dias_atraso ~ C(CAPAG) + Total_Parcelas + Valor_Parcela + C(Tipo_Produto) + C(Tipo_Empregado) + C(Esfera_Convenio) + C(Faixa_Idade) + Muitos_Contratos

                            OLS Regression Results                            
Dep. Variable:        max_dias_atraso   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                     244.5
Date:                Fri, 22 Aug 2025   Prob (F-statistic):               0.00
Time:                        17:51:33   Log-Likelihood:                -36319.
No. Obser

#### Regressão Logística 
Abaixo modelo nos diz que a inadimplência severa é menos sobre o histórico de crédito (`Muitos_Contratos`) e mais sobre o **contexto do cliente e do produto**. O risco é uma combinação de quem o cliente é (idade), onde ele trabalha (esfera do convênio) e o tipo de produto que ele contratou.

Tipo Empregado foi removido por não convergir

In [ ]:

# REGRESSÃO LOGÍSTICA (LOGIT) ---
# OBJETIVO: Entender quais fatores influenciam a PROBABILIDADE de um contrato
#           ter um atraso severo (>= 90 dias).
"""
# obs: estou removendo a variável 'Tipo_Empregado' que causou a falha de convergência
# devido ao problema de separação perfeita, no caso de 90 dias
"""
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

print("="*80)
print("### REFAZENDO A REGRESSÃO LOGÍSTICA SEM A VARIÁVEL 'Tipo_Empregado' ###")
print("="*80)

# Obter preditores e remover 'Tipo_Empregado'
preditores_originais = [col for col in df_sm.columns if col not in ['CCB', 'max_dias_atraso'] + [c for c in df_sm.columns if 'Vencido' in c]]
preditores_corrigidos = [p for p in preditores_originais if p != 'Tipo_Empregado'] #! se for diferente de 90, pode trocar isso

print("\n<DEBUG> Variáveis preditoras utilizadas nesta nova análise:")
print(preditores_corrigidos)

#! ########################################################
target_logit = 'Vencido_90d_Flag'

#! OBSERVAÇÂO: Você pode trocar 90d por 1d, 30d, etc.
#! estou deixando assim apenas para nao ficar muito massivo
#! ########################################################

formula_parts_corrigida = []
for col in preditores_corrigidos:
    if col in df_sm.select_dtypes(include=['object', 'category', 'bool']).columns:
        formula_parts_corrigida.append(f"C({col})")
    else:
        formula_parts_corrigida.append(col)
        
formula_logit_corrigida = f"{target_logit} ~ " + " + ".join(formula_parts_corrigida)

print(f"\n [INFO] fórmula utilizada:\n{formula_logit_corrigida}\n")

model_logit_corrigido = smf.logit(formula=formula_logit_corrigida, data=df_sm).fit()


# SUMÁRIO
print(model_logit_corrigido.summary())

print("\n\n" + "="*80)
print("--- Coeficientes do Modelo (Razão de Chances / Odds Ratio) ---")

odds_ratios_corrigido = pd.DataFrame({
    "Odds Ratio": np.exp(model_logit_corrigido.params),
    "P>|z|": model_logit_corrigido.pvalues
})
conf_int_corrigido = np.exp(model_logit_corrigido.conf_int())
conf_int_corrigido.columns = ['OR CI Inferior', 'OR CI Superior']
odds_ratios_corrigido = odds_ratios_corrigido.join(conf_int_corrigido)

print(odds_ratios_corrigido)

### REFAZENDO A REGRESSÃO LOGÍSTICA SEM A VARIÁVEL 'Tipo_Empregado' ###

<DEBUG> Variáveis preditoras utilizadas nesta nova análise:
['CAPAG', 'Total_Parcelas', 'Valor_Parcela', 'Tipo_Produto', 'Esfera_Convenio', 'Faixa_Idade', 'Muitos_Contratos']

 [INFO] fórmula utilizada:
Vencido_90d_Flag ~ C(CAPAG) + Total_Parcelas + Valor_Parcela + C(Tipo_Produto) + C(Esfera_Convenio) + C(Faixa_Idade) + Muitos_Contratos

Optimization terminated successfully.
         Current function value: 0.611409
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:       Vencido_90d_Flag   No. Observations:                 6838
Model:                          Logit   Df Residuals:                     6825
Method:                           MLE   Df Model:                           12
Date:                Fri, 22 Aug 2025   Pseudo R-squ.:                 0.04151
Time:                        17:56:44   Log-Likelihood:                -4180.8
converged: 

## Referências
Abaixo estão os links que eu visitei para consultar dúvidas: 


machine learning - How to deal with possible data leakage in time series data? - Data Science Stack Exchange
https://datascience.stackexchange.com/questions/45575/how-to-deal-with-possible-data-leakage-in-time-series-data
Python Pandas: Keeping only dataframe rows containing first occurrence of an item - Stack Overflow
https://stackoverflow.com/questions/24136620/python-pandas-keeping-only-dataframe-rows-containing-first-occurrence-of-an-ite/24137060
python - Add column with number of days between dates in DataFrame pandas - Stack Overflow
https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
Python Pandas max value in a group as a new column - Stack Overflow
https://stackoverflow.com/questions/35640364/python-pandas-max-value-in-a-group-as-a-new-column
python - Assign conditions from a list np.select and create a new column (pandas) - Stack Overflow
https://stackoverflow.com/questions/69824116/assign-conditions-from-a-list-np-select-and-create-a-new-column-pandas
python - Pandas get the age from a date (example: date of birth) - Stack Overflow
https://stackoverflow.com/questions/26788854/pandas-get-the-age-from-a-date-example-date-of-birth
Why do we use stratify in train_test_split | by aymuosmukherjee | Medium
https://medium.com/@aymuosmukherjee/why-do-we-use-stratify-in-train-test-split-e3eb296a5494
python - How can I align pandas get_dummies across training / validation / testing? - Stack Overflow
https://stackoverflow.com/questions/56738267/how-can-i-align-pandas-get-dummies-across-training-validation-testing
Feature Engineering in a Pipeline - Weijian Zhang
https://weijianzhg.com/blog/2021/feature-engineering/
Column Transformer with Mixed Types — scikit-learn 1.7.1 documentation
https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
How Does the class_weight Parameter in Scikit-Learn Work? - GeeksforGeeks
https://www.geeksforgeeks.org/machine-learning/how-does-the-classweight-parameter-in-scikit-learn-work/
unbalanced classes - What is the proper usage of scale_pos_weight in xgboost for imbalanced datasets? - Cross Validated
https://stats.stackexchange.com/questions/243207/what-is-the-proper-usage-of-scale-pos-weight-in-xgboost-for-imbalanced-datasets
roc_auc_score — scikit-learn 1.7.1 documentation
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
F1 Score vs ROC AUC vs Accuracy vs PR AUC - Neptune.ai
https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc
python - XGBoostClassifier: model.get_booster().get_score(importance_type="gain") not return entire feature importance - Stack Overflow
https://stackoverflow.com/questions/74067871/xgboostclassifier-model-get-booster-get-scoreimportance-type-gain-not-ret
